In [1]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('order_products_merge.csv')
data = data.fillna(0)
print(data.shape)
data.head()

(595542, 507)


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Raw Coconut Water,100% Recycled Paper Towels,...,YoBaby Blueberry Apple Yogurt,YoBaby Peach Pear Yogurt,YoKids Blueberry & Strawberry/Vanilla Yogurt,YoKids Squeeze! Organic Strawberry Flavor Yogurt,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry",Yobaby Organic Plain Yogurt,"Yogurt, Lowfat, Strawberry","Yogurt, Strained Low-Fat, Coconut",ZBar Organic Chocolate Brownie Energy Snack,Crackers Cheddar Bunnies Snack Packs
0,4,178520,36,1,9,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,156122,42,6,16,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,54901,51,3,11,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,18,118860,3,4,20,6.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,98256,29,3,13,6.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
users = list(data['user_id'].unique())
len(users)

10000

In [4]:
orders = list(data['order_id'].unique())
len(orders)

594798

In [5]:
top_products = data.drop(columns = ['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_number', 'user_id', 'order_id']).sum(axis=0)
top_products = list(top_products.sort_values(ascending=False)[:50].index)
# top_products.remove('Others')

In [8]:
def process(row):
    user_orders = data[data['user_id'] == row['user_id']]
        
    organics = [i for i in row.index if 'organic' in str(i).lower() ]
    organics = sum(row[organics])
    
    global_reorders = [i for i in row.index if i in top_products]
    global_reorders = sum(row[global_reorders])
    
    basket_size = sum(row.drop(['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_number', 'user_id', 'order_id']).values)
    num_orders = len(user_orders)        
    
    if row['order_number'] > min(user_orders['order_number'].values):
        prev_orders_inc = user_orders[user_orders['order_number']<=row['order_number']]
        
        days_since_first_order = max(prev_orders_inc['days_since_prior_order'].cumsum())
        
        reorders = prev_orders_inc.drop(columns=['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_number', 'user_id', 'order_id']).sum(axis=0)
        reorders = sum([1 if r > 1 else 0 for r in list(reorders.values) ])
        
        if (row['days_since_prior_order'] == 7) and (reorders/basket_size > 0.5):
            tx_purpose_weekly = 1
            tx_purpose_monthly = 0
        elif (row['days_since_prior_order'] == 30) and (reorders/basket_size > 0.5):
            tx_purpose_monthly = 1
            tx_purpose_weekly = 0
        else:
            tx_purpose_weekly = 0
            tx_purpose_monthly = 0
    else:
        days_since_first_order = 0
        reorders = 0
        tx_purpose_weekly = 0
        tx_purpose_monthly = 0
    
    return pd.Series([days_since_first_order, reorders, organics, basket_size, num_orders, global_reorders, tx_purpose_weekly, tx_purpose_monthly])

In [9]:
data[['days_since_first_order', 'reorders', 'organics', 'basket_size', 'num_orders', 'global_reorders', 'tx_purpose_weekly', 'tx_purpose_monthly']] = data.progress_apply(process, axis=1)

100%|█████████████████████████████████████████████████████████████████████████| 595542/595542 [56:07<00:00, 176.87it/s]


In [10]:
print(data.shape)
data.head()

(595542, 515)


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Raw Coconut Water,100% Recycled Paper Towels,...,ZBar Organic Chocolate Brownie Energy Snack,Crackers Cheddar Bunnies Snack Packs,days_since_first_order,reorders,organics,basket_size,num_orders,global_reorders,tx_purpose_weekly,tx_purpose_monthly
0,4,178520,36,1,9,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,231.0,10.0,0.0,13.0,58.0,12.0,1.0,0.0
1,5,156122,42,6,16,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,289.0,13.0,3.0,26.0,53.0,24.0,0.0,0.0
2,15,54901,51,3,11,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,152.0,19.0,1.0,5.0,99.0,3.0,0.0,0.0
3,18,118860,3,4,20,6.0,0.0,0.0,0.0,1.0,...,0.0,0.0,13.0,13.0,6.0,28.0,47.0,18.0,0.0,0.0
4,28,98256,29,3,13,6.0,1.0,0.0,0.0,0.0,...,0.0,0.0,128.0,32.0,7.0,16.0,81.0,8.0,0.0,0.0


In [11]:
data.to_csv('final_data.csv')